# Running DeepSpeedChaton Amazon SageMaker

This is a sample code to run stanford_alpaca on Amazon SageMaker, for demo or research use only!

In [3]:
## Update sagemaker python sdk version
!pip install -U sagemaker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 886.2/886.2 kB 44.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.187.0-py2.py3-none-any.whl size=1186949 sha256=528702d588be2c57879c15c3330d4f1f34f40cffe67b855b599b68e431fa3fdb
  Stored in directory: /home/ec2-user/.cache/pip/wheels/21/d2/2b/ed240a9d3084c5b4e3e7fd2ca8f9c11659bb98f0b87b6b1ca3
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.186.0
    Uninstalling sagemaker-2.186.0:
      Successfully uninstalled sagemaker-2.186.0


In [4]:
import sagemaker
import boto3
from sagemaker import get_execution_role

sess = sagemaker.Session()
role = get_execution_role()
sagemaker_default_bucket = sess.default_bucket()
print(sagemaker_default_bucket)

account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker-us-east-1-427169985960


## Download pretrained model from HuggingFace Hub

To avoid download model from Huggingface hub failure, we download first and push those model files to S3 bucket first.

In [5]:
!pip install huggingface_hub

In [23]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_cache_path = Path("./model")
local_cache_path.mkdir(exist_ok=True)

#model_name = "TheBloke/Llama-2-13B-fp16"
model_name = "TheBloke/Llama-2-7B-fp16"

# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.model"]

model_download_path = snapshot_download(
    repo_id=model_name,
    cache_dir=local_cache_path,
    allow_patterns=allow_patterns,
)

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

**Upload model files to S3**

In [24]:
# Get the model files path
import os
from glob import glob

local_model_path = None

paths = os.walk(r'./model')

for root, dirs, files in paths:
    for file in files:
        print(file)
        if file == 'config.json':
            print(os.path.join(root,file))
            local_model_path = str(os.path.join(root,file))[0:-11]
            print(local_model_path)
if local_model_path == None:
    print("Model download may failed, please check prior step!")

main
9e556afd44213b6bd1be2b850ebbbd98f5481437a8021afaf58ee7fb1818d347
400e3de6ffc3884ec3c158a046f6a04da00ef3ca
4cad8f6ae78aa8cf03e750a83a2b42f7ecc03c54
422bfd1b447eb0182551ac305fbc03e299e7f37a981317590e3a4362b6708179
77b916244c2215098baa37dbdee4037698fc31744dff92209d3142214294fce9
db7264b24cac7a39947bb5fc02fe5c2d7ac9eaf4
67a2e09f9d8b5e85eca24e88aa5c0fb465bbafd6
d85ba6cb6820b01226ef8bd40b46bb489041c6a8
89c31b8e044c6d3589cd86fe3be48558d8bab8fc
tokenizer.model
pytorch_model-00002-of-00002.bin
tokenizer.json
pytorch_model.bin.index.json
generation_config.json
pytorch_model-00001-of-00002.bin
special_tokens_map.json
tokenizer_config.json
config.json
./model/models--TheBloke--Llama-2-7B-fp16/snapshots/ba2306439903c2ebf7d09970a973ef44d1402239/config.json
./model/models--TheBloke--Llama-2-7B-fp16/snapshots/ba2306439903c2ebf7d09970a973ef44d1402239/


### Download s5cmd and put it in src folder

In [ ]:
!wget https://github.com/peak/s5cmd/releases/download/v2.2.2/s5cmd_2.2.2_Linux-64bit.tar.gz

!tar -xvf s5cmd_2.2.2_Linux-64bit.tar.gz

!mv s5cmd src/
!rm -rf s5cmd_2.2.2_Linux-64bit.tar.gz

**Rewrite upload module**

In [29]:
%%script env sagemaker_default_bucket=$sagemaker_default_bucket local_model_path=$local_model_path bash

chmod +x src/s5cmd
src/s5cmd sync ${local_model_path} s3://${sagemaker_default_bucket}/Llama-2-7B-fp16/

cp model/models--TheBloke--Llama-2-7B-fp16/snapshots/ba2306439903c2ebf7d09970a973ef44d1402239/generation_config.json s3://sagemaker-us-east-1-427169985960/Llama-2-7B-fp16/generation_config.json
cp model/models--TheBloke--Llama-2-7B-fp16/snapshots/ba2306439903c2ebf7d09970a973ef44d1402239/config.json s3://sagemaker-us-east-1-427169985960/Llama-2-7B-fp16/config.json
cp model/models--TheBloke--Llama-2-7B-fp16/snapshots/ba2306439903c2ebf7d09970a973ef44d1402239/pytorch_model.bin.index.json s3://sagemaker-us-east-1-427169985960/Llama-2-7B-fp16/pytorch_model.bin.index.json
cp model/models--TheBloke--Llama-2-7B-fp16/snapshots/ba2306439903c2ebf7d09970a973ef44d1402239/tokenizer_config.json s3://sagemaker-us-east-1-427169985960/Llama-2-7B-fp16/tokenizer_config.json
cp model/models--TheBloke--Llama-2-7B-fp16/snapshots/ba2306439903c2ebf7d09970a973ef44d1402239/special_tokens_map.json s3://sagemaker-us-east-1-427169985960/Llama-2-7B-fp16/special_tokens_map.json
cp model/models--TheBloke--Llama-2-7B-fp

## Prepare a docker image

In [95]:
%%writefile Dockerfile
## You should change below region code to the region you used, here sample is use us-west-2
#From 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-training:1.13.1-transformers4.26.0-gpu-py39-cu117-ubuntu20.04 
#From 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-training:1.13.1-transformers4.26.0-gpu-py39-cu117-ubuntu20.04 
#From 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-training:2.0.0-transformers4.28.1-gpu-py310-cu118-ubuntu20.04
FROM nvcr.io/nvidia/pytorch:23.02-py3
RUN pip3 install sagemaker-training

#Remove Cuda 11.8
#RUN apt-get -y purge cuda*
#RUN apt-get -y autoremove
#RUN apt-get -y autoclean
#RUN rm -rf /usr/local/cuda*

#install Cuda 12
#RUN wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-ubuntu2004.pin
#RUN mv cuda-ubuntu2004.pin /etc/apt/preferences.d/cuda-repository-pin-600
#RUN wget https://developer.download.nvidia.com/compute/cuda/12.1.1/local_installers/cuda-repo-ubuntu2004-12-1-local_12.1.1-530.30.02-1_amd64.deb
#RUN dpkg -i cuda-repo-ubuntu2004-12-1-local_12.1.1-530.30.02-1_amd64.deb
#RUN cp /var/cuda-repo-ubuntu2004-12-1-local/cuda-*-keyring.gpg /usr/share/keyrings/
#RUN apt-get update
#RUN apt-get -y install cuda

ENV LANG=C.UTF-8
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE

RUN update-alternatives --display cuda
RUN update-alternatives --auto cuda
RUN python3 -m pip uninstall -y deepspeed 
#RUN python3 -m pip install pytorch-lightning==1.9.0

#From requirement.txt
RUN python3 -m pip install datasets>=2.8.0
RUN python3 -m pip install sentencepiece>=0.1.97
RUN python3 -m pip install protobuf==3.20.3
RUN python3 -m pip install accelerate>=0.15.0
RUN python3 -m pip install torch>=1.12.0
RUN python3 -m pip install deepspeed>=0.9.0
RUN python3 -m pip install transformers==4.31.0
#RUN python3 -m pip install transformers>=4.31.0,!=4.33.2
RUN python3 -m pip install tensorboard


## Install transfomers version which support LLaMaTokenizer
#RUN python3 -m pip install git+https://github.com/huggingface/transformers.git@68d640f7c368bcaaaecfc678f11908ebbd3d6176

## Make all local GPUs visible
ENV NVIDIA_VISIBLE_DEVICES="all"

Overwriting Dockerfile


In [6]:
## You should change below region code to the region you used, here sample is use us-east-1
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


**Build image and push to ECR.**

In [9]:
## define repo name, should contain *sagemaker* in the name
repo_name = "sagemaker-llama2-13b"

In [98]:
%%script env repo_name=$repo_name bash

#!/usr/bin/env bash

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.
# The name of our algorithm

algorithm_name=${repo_name}

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} . 
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded
Sending build context to Docker daemon  113.5MB
Step 1/17 : FROM nvcr.io/nvidia/pytorch:23.02-py3
 ---> 7c3375e220ea
Step 2/17 : RUN pip3 install sagemaker-training
 ---> Using cache
 ---> c26db60cc80d
Step 3/17 : ENV LANG=C.UTF-8
 ---> Using cache
 ---> daf94321439e
Step 4/17 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> bd8d2fd7a9a1
Step 5/17 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> ff65d34542c3
Step 6/17 : RUN update-alternatives --display cuda
 ---> Using cache
 ---> e04ac8360543
Step 7/17 : RUN update-alternatives --auto cuda
 ---> Using cache
 ---> 6c2f3c2edc0f
Step 8/17 : RUN python3 -m pip uninstall -y deepspeed
 ---> Using cache
 ---> f14a5a8f5df8
Step 9/17 : RUN python3 -m pip install datasets>=2.8.0
 ---> Using cache
 ---> 6065025ed0e2
Step 10/17 : RUN python3 -m pip install sentencepiece>=0.1.97
 ---> Using cache
 ---> e9186d65012f
Step 11/17 : RUN python3 -m pip install protobuf==3.20.3
 ---> Using cache
 ---> ee7e2bcf85e7
Step 12/17

In [16]:
!pwd

/home/ec2-user/SageMaker/DeepSpeedExamples/applications/DeepSpeed-Chat


In [91]:
!service docker stop
!service docker start

Redirecting to /bin/systemctl stop docker.service
Failed to stop docker.service: The name org.freedesktop.PolicyKit1 was not provided by any .service files
See system logs and 'systemctl status docker.service' for details.
  docker.socket
Redirecting to /bin/systemctl start docker.service
Failed to start docker.service: The name org.freedesktop.PolicyKit1 was not provided by any .service files
See system logs and 'systemctl status docker.service' for details.


### Generate training entrypoint script

**Note: DO NOT CHANGE BELOW VAlUE OF "output_dir" and "cache_dir", keep it "/tmp/llama_out" and "/tmp".**

Below is just a testing to fine-tune on a sample dataset (just 8 samples), you could change ```data_path``` to your dataset for furthur fine tune.

For the dataset download, you could follow the way how to download pretrain model:
```
./s5cmd sync s3://$MODEL_S3_BUCKET/llama/pretrain/7B/* /tmp/llama_pretrain/
```

It is recommend to use the folder ```/tmp/dataset/```.

## Modify some files due to some bugs

In [ ]:
#modify DeepSpeedChat-on-Amazon-SageMaker/src/DeepSpeed-Chat/training/utils/utils.py

#Find below function
def load_hf_tokenizer(model_name_or_path, fast_tokenizer=True):
    if os.path.exists(model_name_or_path):
        # Locally tokenizer loading has some issue, so we need to force download
        model_json = os.path.join(model_name_or_path, "config.json")
        if os.path.exists(model_json):
            model_json_file = json.load(open(model_json))
            model_name = model_json_file["_name_or_path"]
            tokenizer = get_tokenizer(model_name,
                                      fast_tokenizer=fast_tokenizer)
    else:
        tokenizer = get_tokenizer(model_name_or_path,
                                  fast_tokenizer=fast_tokenizer)

    return tokenizer

#####
#Replace it into

def load_hf_tokenizer(model_name_or_path, fast_tokenizer=True):
    tokenizer = get_tokenizer(model_name_or_path,
                                  fast_tokenizer=fast_tokenizer)

    return tokenizer


## Make some sample training Data (Optional)

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_cache_path = Path("./dataset")
local_cache_path.mkdir(exist_ok=True)

model_name = "Dahoas/synthetic-instruct-gptj-pairwise"#

# Only download pytorch checkpoint files
allow_patterns = ["*"]

model_download_path = snapshot_download(
    repo_id=model_name,
    cache_dir=local_cache_path,
    allow_patterns=allow_patterns,
)

### Modify training script

In [ ]:
%%writefile src/train.sh
#!/bin/bash

chmod +x ./s5cmd
./s5cmd sync s3://$MODEL_S3_BUCKET/Llama-2-7B-fp16/* /tmp/llama-2-7B-fp16/
#./s5cmd sync s3://$MODEL_S3_BUCKET/llama-7b-hf/* /tmp/llama-7b-hf/


cd DeepSpeed-Chat/training/step1_supervised_finetuning/

#Fix huggingface issue
#pip install --upgrade huggingface_hub
#huggingface-cli login --token hf_xxxx


#bash training_scripts/opt/single_gpu/run_1.3b.sh
#!/bin/bash
# Copyright (c) Microsoft Corporation.
# SPDX-License-Identifier: Apache-2.0

# DeepSpeed Team
OUTPUT=$1
ZERO_STAGE=$2
if [ "$OUTPUT" == "" ]; then
    OUTPUT=./output_step1_llama2_7b_sft_demo
fi
if [ "$ZERO_STAGE" == "" ]; then
    ZERO_STAGE=3
fi
mkdir -p $OUTPUT

deepspeed main.py \
   --data_path local/jsonfile \
   --data_split 10,0,0 \
   --model_name_or_path /tmp/llama-2-7B-fp16/ \
   --per_device_train_batch_size 2 \
   --per_device_eval_batch_size 2 \
   --max_seq_len 512 \
   --learning_rate 9.65e-6 \
   --weight_decay 0. \
   --num_train_epochs 2  \
   --gradient_accumulation_steps 1 \
   --lr_scheduler_type cosine \
   --num_warmup_steps 0 \
   --seed 1234 \
   --gradient_checkpointing \
   --zero_stage $ZERO_STAGE \
   --deepspeed \
   --output_dir $OUTPUT \
   --print_loss
   
#   --model_name_or_path TheBloke/Llama-2-13B-fp16 \
#   --model_name_or_path /tmp/Llama-2-13B-fp16/ \
#&> $OUTPUT/training.log

#set the timer for stopping the training job
#sleep 900
#kill "$!"

if [ $? -eq 1 ]; then
    echo "Training script error, please check CloudWatch logs"
    exit 1
fi

#Upload the trained models to S3
./s5cmd sync $OUTPUT s3://$MODEL_S3_BUCKET/deepspeedchat-finetuned-llama-2-7B-fp16-20230928

#s3://$MODEL_S3_BUCKET/RWKV/output/$(date +%Y-%m-%d-%H-%M-%S)/

In [10]:
## The image uri which is build and pushed above
image_uri = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account, region, repo_name)
image_uri

'427169985960.dkr.ecr.us-east-1.amazonaws.com/sagemaker-llama2-13b:latest'

**The modified training script**

Everything is ready, let's launch the training job.

## Create SageMaker Training Job

In [27]:
!sudo chmod 777 lost+found

chmod: cannot access ‘lost+found’: No such file or directory


In [55]:
!sudo chmod -R 777 ./src

In [ ]:
import time
from sagemaker.estimator import Estimator

environment = {
              'MODEL_S3_BUCKET': sagemaker_default_bucket # The bucket to store pretrained model and fine-tune model
}

base_job_name = 'DeepSpeedChat-SageMaker-Training'


#instance_type = 'ml.p3dn.24xlarge'
#instance_type = 'ml.p4d.24xlarge'
instance_type = 'ml.g5.12xlarge'

estimator = Estimator(role=role,
                      entry_point='train.sh',
                      source_dir='./src',
                      base_job_name=base_job_name,
                      instance_count=1,
                      instance_type=instance_type,
                      image_uri=image_uri,
                      environment=environment,
                      disable_profiler=True,
                      debugger_hook_config=False,
                      keep_alive_period_in_seconds=3600 #make sure you have quota for SageMaker warm pool, it's easier for debuging
                      )

estimator.fit()
# estimator.fit(inputs)